In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from graphcast import data_utils
import dataclasses
from graphcast import graphcast
import his_utils

In [2]:
file_name = "testdata/source-era5_date-2022-01-01_res-0.25_levels-37_steps-12.nc"
eval_data = xr.open_dataset(file_name)
print(eval_data)

<xarray.Dataset> Size: 13GB
Dimensions:                       (lon: 1440, lat: 721, level: 37, time: 14,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 6kB 0.0 0.25 0.5 ... 359.5 359.8
  * lat                           (lat) float32 3kB -90.0 -89.75 ... 89.75 90.0
  * level                         (level) int32 148B 1 2 3 5 ... 950 975 1000
  * time                          (time) timedelta64[ns] 112B 0 days 00:00:00...
    datetime                      (batch, time) datetime64[ns] 112B ...
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 4MB ...
    land_sea_mask                 (lat, lon) float32 4MB ...
    2m_temperature                (batch, time, lat, lon) float32 58MB ...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 58MB ...
    10m_v_component_of_wind       (batch, time, lat, lon) float32 58MB ...
    10m_u_component_of_wind       (b

In [3]:
task_config = graphcast.TaskConfig(
        input_variables=(graphcast.TARGET_SURFACE_VARS + graphcast.TARGET_ATMOSPHERIC_VARS + graphcast.FORCING_VARS +
        graphcast.STATIC_VARS),
        target_variables=graphcast.TARGET_SURFACE_VARS + graphcast.TARGET_ATMOSPHERIC_VARS,
        forcing_variables=graphcast.FORCING_VARS,
        pressure_levels=graphcast.PRESSURE_LEVELS[37],
        input_duration="6h"
    )

eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    eval_data, 
    target_lead_times=slice("6h", f"{10 * 6}h"), 
    **dataclasses.asdict(task_config)
)
eval_targets = eval_targets * np.nan

In [4]:
sample1 = xr.open_dataset("testdata/1.nc")
sample2 = xr.open_dataset("testdata/2.nc")

In [5]:
sample1["z"].loc[{"level":1000}]

<xarray.DataArray 'z' (time: 24, latitude: 721, longitude: 1440)> Size: 199MB
[24917760 values with dtype=float64]
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
    level      int32 4B 1000
  * time       (time) datetime64[ns] 192B 2022-01-01 ... 2022-01-01T23:00:00
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [28]:
sample2

<xarray.Dataset> Size: 2GB
Dimensions:    (longitude: 1440, latitude: 721, time: 24)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 192B 2022-01-01 ... 2022-01-01T23:00:00
Data variables:
    u10        (time, latitude, longitude) float64 199MB ...
    v10        (time, latitude, longitude) float64 199MB ...
    t2m        (time, latitude, longitude) float64 199MB ...
    z          (time, latitude, longitude) float64 199MB ...
    lsm        (time, latitude, longitude) float64 199MB ...
    msl        (time, latitude, longitude) float64 199MB ...
    tisr       (time, latitude, longitude) float64 199MB ...
    tp         (time, latitude, longitude) float64 199MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-11 06:28:22 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [7]:
# his_utils.compare_dataarrays(sample1["z"].loc[{"level":1000}], sample2["z"])

In [29]:
sample2 = sample2.drop_vars("z")

In [30]:
sample = xr.merge([sample1, sample2])

In [31]:
sample

<xarray.Dataset> Size: 46GB
Dimensions:    (longitude: 1440, latitude: 721, level: 37, time: 24)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) int32 148B 1 2 3 5 7 10 20 ... 875 900 925 950 975 1000
  * time       (time) datetime64[ns] 192B 2022-01-01 ... 2022-01-01T23:00:00
Data variables: (12/13)
    z          (time, level, latitude, longitude) float64 7GB ...
    q          (time, level, latitude, longitude) float64 7GB ...
    t          (time, level, latitude, longitude) float64 7GB ...
    u          (time, level, latitude, longitude) float64 7GB ...
    v          (time, level, latitude, longitude) float64 7GB ...
    w          (time, level, latitude, longitude) float64 7GB ...
    ...         ...
    v10        (time, latitude, longitude) float64 199MB ...
    t2m        (time, latitude, longitude) float64 199MB ...
    lsm        (time, latitude, longitude) float64 199MB ...
    msl        (time, latitude, longitude) float64 199MB ...
    tisr       (time, latitude, longitude) float64 199MB ...
    tp         (time, latitude, longitude) float64 199MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-11 06:25:23 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...